<a href="https://colab.research.google.com/github/Yuliya3003/Parser/blob/main/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Устанавливаем зависимости

In [ ]:
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'

Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver

# Install Selenium
pip install selenium

### Импортируем используемые модули

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import csv

service = Service("/usr/bin/chromedriver")
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)

### Методы, которые сохраняют название статьи, автора, и ссылку

In [3]:
def get_titles(driver):
  titles = driver.find_elements(By.CSS_SELECTOR, "li h2")
  titles = [title.text for title in titles]
  return titles

def get_authors(driver):
  authors = driver.find_elements(By.CSS_SELECTOR, "#search-results li span")
  authors = [author.text for author in authors[::2]] # Есть несколько span'ов без класса, поэтому берем только каждый третий
  return authors

def get_urls(driver):
  urls = driver.find_elements(By.CSS_SELECTOR, "li h2 a")
  urls = [url.get_attribute('href') for url in urls]
  return urls



### Метод, который находит всех авторов, названия и ссылки на одной странице (по умолчанию 1)

In [4]:
def make_query_one_page(query, page=1):

  query = query.replace(" ", "%20") # подгоняем url под кодировку, в которой пробелы представляют из себя %20

  try:
    url = f"https://cyberleninka.ru/search?q={query}&page={page}"

    driver.get(url)
    time.sleep(5) # Дожидаемся загрузки страницы

    titles = get_titles(driver)
    authors = get_authors(driver)
    urls = get_urls(driver)

    driver.quit()

  except:
    return None

  return titles, authors, urls

### Метод, который проходится по всем страницам

In [5]:
def make_query_all_possible_pages(query):

  query = query.replace(" ", "%20") # подгоняем url под кодировку, в которой пробелы представляют из себя %20

  titles = []
  authors = []
  urls = []
  page = 0

  while True:
    time.sleep(5)
    page += 1

    try:
      url = f"https://cyberleninka.ru/search?q={query}&page={page}"

      driver.get(url)
      time.sleep(5) # Дожидаемся загрузки страницы

      titles += get_titles(driver)
      authors += get_authors(driver)
      urls += get_urls(driver)

      driver.quit()

    except:
      break

  return titles, authors, urls

### Метод, который сохраняет информацию в CSV файл

In [6]:
def csv_writer(data, path):
    """
    Write data to a CSV file path
    """
    with open(path, "w", newline='') as csv_file:
          writer = csv.writer(csv_file, delimiter=',')
          for line in data:
              writer.writerow(line)

### Основной блок

In [7]:
query = "микросервисная архитектура"
res = make_query_one_page(query)
data = ["url,author,title".split(",")]
for index in range(len(res[0])):
  data.append([res[2][index], res[1][index], res[0][index]])

path = "output.csv"
csv_writer(data, path)